# Sentiment Analisys with Recursive Neural Network


In [ ]:
!curl https://colab.chainer.org/install | sh -

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1580  100  1580    0     0   9132      0 --:--:-- --:--:-- --:--:--  9080
+ apt -y -q install cuda-libraries-dev-10-0
Reading package lists...
Building dependency tree...
Reading state information...
cuda-libraries-dev-10-0 is already the newest version (10.0.130-1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
+ pip install -q cupy-cuda100  chainer 
     |████████████████████████████████| 369.7MB 46kB/s 
+ set +ex
Installation succeeded!


Let's import the necessary modules, then check the version of Chainer, NumPy, CuPy, Cuda and other execution environments.

In [ ]:
import collections
import numpy as np

import chainer
from chainer import cuda
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer import reporter


chainer.print_runtime_info()

/usr/local/lib/python3.6/dist-packages/chainer/backends/cuda.py:147: UserWarning: cuDNN is not enabled.
Please reinstall CuPy after you install cudnn
(see https://docs-cupy.chainer.org/en/stable/install.html#install-cudnn).
  'cuDNN is not enabled.\n'


Platform: Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
Chainer: 7.4.0
ChainerX: Not Available
NumPy: 1.18.5
CuPy:
  OS                           : Linux-4.19.112+-x86_64-with-Ubuntu-18.04-bionic
  CuPy Version                 : 8.2.0
  NumPy Version                : 1.18.5
  SciPy Version                : 1.4.1
  Cython Build Version         : 0.29.21
  CUDA Root                    : /usr/local/cuda
  CUDA Build Version           : 10000
  CUDA Driver Version          : 10010
  CUDA Runtime Version         : 10000
  cuBLAS Version               : 10000
  cuFFT Version                : 10000
  cuRAND Version               : 10000
  cuSOLVER Version             : (10, 0, 0)
  cuSPARSE Version             : 10000
  NVRTC Version                : (10, 0)
  Thrust Version               : 100903
  CUB Build Version            : 100800
  cuDNN Build Version          : 7605
  cuDNN Version                : 7605
  NCCL Build Version           : 2604
  NCCL Runtime Version         : 2604
  cu

### 2. Setting parameters
Here we set the parameters for training.
* `` n_epoch``: Epoch number. How many times we pass through the whole training data.
* `` n_units``: Number of units. How many hidden state vectors each Recursive Neural Network node has.
* `` batchsize``: Batch size. How many train data we will input as a block when updating parameters.
* `` n_label``: Number of labels. Number of classes to be identified. Since there are 5 labels this time, `` 5``.
* `` epoch_per_eval``: How often to perform validation.
* `` is_test``: If `` True``, we use a small dataset.
* `` gpu_id``: GPU ID. The ID of the GPU to use. For Colaboratory it is good to use `` 0``.

In [ ]:
# parameters
n_epoch = 25  # number of epochs
n_units = 30  # number of units per layer
batchsize = 1  # minibatch size
n_label = 2  # number of labels
epoch_per_eval = 2 # number of epochs per evaluation
is_test = False
gpu_id = 1

if is_test:
    max_size = 10
else:
    max_size = None

### 3. Preparing the iterator

Let's read the dataset used for training, validation, test and create an Iterator.

First, we convert each sample represented by ``str`` type to a tree structure data represented by a ``dictionary`` type.

We will tokenize the string with `` read_corpus`` implemented by the parser `` SexpParser``. After that, we convert each tokenized sample to a tree structure data  by `` convert_tree``. By doing like this, it is possible to express a label as ``int``, a node as a two-element ``tuple``, and a tree structure as a ``dictionary``, making it a more manageable data structure than the original string.


In [ ]:
# data.py
import codecs
import re


class SexpParser(object):

    def __init__(self, line):
        self.tokens = re.findall(r'\(|\)|[^\(\) ]+', line)
        self.pos = 0
        

    def parse(self):
        assert self.pos < len(self.tokens)
        token = self.tokens[self.pos]
        assert token != ')'
        self.pos += 1
        
        
        if token == '(':
            children = []
            while True:
                assert self.pos < len(self.tokens)
                if self.tokens[self.pos] == ')':
                    self.pos += 1
                    break
                else:
                    children.append(self.parse())            
            return children
        else:
            return token


def read_corpus(path, max_size):
    with codecs.open(path, encoding='utf-8') as f:
        trees = []
        for line in f:
            line = line.strip()
            tree = SexpParser(line).parse()
            trees.append(tree)
            if max_size and len(trees) >= max_size:
                break
    
    return trees

def convert_tree(vocab, exp):
    assert isinstance(exp, list) and (len(exp) == 2 or len(exp) == 3)
    if len(exp) == 2:
        label, leaf = exp
        if leaf not in vocab:
            vocab[leaf] = len(vocab)
        return {'label': int(label), 'node': vocab[leaf]}
    elif len(exp) == 3:
        label, left, right = exp
       # f.write('new node'+"\n")
        node = (convert_tree(vocab, left), convert_tree(vocab, right))
       # print(exp)
       # f.write('new node'+"\n")
        return {'label': int(label), 'node': node}

def convert_tree2(vocab, exp, f):
    assert isinstance(exp, list) and (len(exp) == 2 or len(exp) == 3)
    if len(exp) == 2:
        label, leaf = exp
        if leaf not in vocab:
            vocab[leaf] = len(vocab)
        return {'label': int(label), 'node': vocab[leaf]}
    elif len(exp) == 3:
        label, left, right = exp
       # f.write('new node'+"\n")
        node = (convert_tree(vocab, left, f), convert_tree(vocab, right, f))
       # print(exp)
        f.write('new node'+"\n")
        return {'label': int(label), 'node': node}

Let's use `` read_corpus () `` and `` convert_tree () `` to create an iterator.

In [ ]:

vocab = {}

train_data = [convert_tree(vocab, tree)
                        for tree in read_corpus('/content/train.txt', max_size)]
train_iter = chainer.iterators.SerialIterator(train_data, batchsize)

validation_data = [convert_tree(vocab, tree)
                                 for tree in read_corpus('/content/dev.txt', max_size)]
validation_iter = chainer.iterators.SerialIterator(validation_data, batchsize,
                                                                                   repeat=False, shuffle=False)

test_data = [convert_tree(vocab, tree)
                        for tree in read_corpus('/content/test.txt', max_size)]





Let's try to display the first element of `` test_data``. It is represented by the following tree structure, `` lable`` expresses the score of that `` node``, and the numerical value of the leaf `` node`` corresponds to the word id in the dictionary `` vocab``.


### 4. Preparing the model

Let's define the network.

We traverse each node of the tree structure data by `` traverse`` and calculate the loss `` loss`` of the whole tree. The implementation of `` traverse`` is a recursive call, which will traverse child nodes in turn. (It is a common implementation when treating tree structure data!)

First, we calculate the hidden state vector `` v``. In the case of a leaf node, we obtain a hidden state vector stored in `` embed`` by `` model.leaf(word) `` from word id`` word``. In the case of an intermediate node, the hidden vector is calculated with the hidden state vector `` left`` and `` right`` of the child nodes by `` v = model.node(left, right)``.

`` loss += F.softmax_cross_entropy(y, t) `` adds the loss of the current node to the loss of the child node, then returns loss to the parent node by ``return loss, v``.

After the line `` loss += F.softmax_cross_entropy(y, t)``, there are some lines for logging accuracy and etc. But it is not necessary for the model definition itself.

In [ ]:
class RecursiveNet(chainer.Chain):
    
    def traverse(self, node, evaluate=None, root=True):
        if isinstance(node['node'], int):
            # leaf node
            word = self.xp.array([node['node']], np.int32)
            loss = 0
            v = model.leaf(word)
        else:
            # internal node
            left_node, right_node = node['node']
            left_loss, left = self.traverse(left_node, evaluate=evaluate, root=False)
            right_loss, right = self.traverse(right_node, evaluate=evaluate, root=False)
            v = model.node(left, right)
            loss = left_loss + right_loss

        y = model.label(v)

        label = self.xp.array([node['label']], np.int32)
        t = chainer.Variable(label)
        loss += F.softmax_cross_entropy(y, t)

        predict = cuda.to_cpu(y.data.argmax(1))
        if predict[0] == node['label']:
            evaluate['correct_node'] += 1
        evaluate['total_node'] += 1
  
        if root:
            if predict[0] == node['label']:
                evaluate['correct_root'] += 1
            evaluate['total_root'] += 1

        return loss, v

    def __init__(self, n_vocab, n_units):
        super(RecursiveNet, self).__init__()
        with self.init_scope():
            self.embed = L.EmbedID(n_vocab, n_units)
            self.l = L.Linear(n_units * 2, n_units)
            self.w = L.Linear(n_units, n_label)

    def leaf(self, x):
        return self.embed(x)

    def node(self, left, right):
        return F.tanh(self.l(F.concat((left, right))))

    def label(self, v):
        return self.w(v)
    
    def __call__(self, x):
        accum_loss = 0.0
        result = collections.defaultdict(lambda: 0)
        for tree in x:
            loss, _ = self.traverse(tree, evaluate=result)
            accum_loss += loss
        
        reporter.report({'loss': accum_loss}, self)
        reporter.report({'total': result['total_node']}, self)
        reporter.report({'correct': result['correct_node']}, self)
        return accum_loss

One attention to the implementation of `` __call__``.

`` x`` passed to `` __call__`` is mini-batched input data and contains samples `` s_n`` like `` [s_1, s_2, ..., s_N] ``.

In a network such as Convolutional Network used for image recognition, it is possible to perform parallel calculation collectively for mini batch `` x``. However, in the case of a tree-structured network like this one, it is difficult to compute parallel because of the following reasons.

* Data length varies depending on samples.
* The order of calculation for each sample is different.

So, the implementation is to calculate each sample and finally summarize the results.
 
Note: Actually, you can perform parallel calculation of mini batch in Recursive Neural Network by using stack. Since it is published in the latter part of notebook as (Advanced), please refer to it.


In [ ]:
model = RecursiveNet(len(vocab), n_units)

if gpu_id >= 0:
   model.to_gpu()

# Setup optimizer
optimizer = chainer.optimizers.AdaGrad(lr=0.01)
optimizer.setup(model)
optimizer.add_hook(chainer.optimizer_hooks.WeightDecay(0.0001))

### 5. Preparation and training of Updater · Trainer

As usual, we define an updater and a trainer to train the model.
This time, I do not use `` L.Classifier`` and calculate the accuracy `` accuracy`` by myself. You can easily implement it using `` extensions.MicroAverage``. For details, please refer to [chainer.training.extensions.MicroAverage](https://docs.chainer.org/en/latest/reference/generated/chainer.training.extensions.MicroAverage.html).

In [ ]:
def _convert(batch, device):
  return batch

import matplotlib
matplotlib.use('Agg')

updater = chainer.training.StandardUpdater(
    train_iter, optimizer, device=gpu_id, converter=_convert)

trainer = chainer.training.Trainer(updater, (n_epoch, 'epoch'))
trainer.extend(
        extensions.Evaluator(validation_iter, model, device=gpu_id, converter=_convert),
        trigger=(epoch_per_eval, 'epoch'))
trainer.extend(extensions.LogReport())

trainer.extend(extensions.MicroAverage(
        'main/correct', 'main/total', 'main/accuracy'))
trainer.extend(extensions.MicroAverage(
        'validation/main/correct', 'validation/main/total',
        'validation/main/accuracy'))

trainer.extend(extensions.PrintReport(
        ['epoch', 'main/loss', 'validation/main/loss', 
          'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

trainer.extend(
    extensions.PlotReport(['main/loss', 'validation/main/loss'],
                          'epoch', filename='loss.png'))

trainer.extend(
    extensions.PlotReport(['main/accuracy', 'validation/main/accuracy'],
                          'epoch', filename='accuracy.png'))


trainer.run()

/usr/local/lib/python3.6/dist-packages/chainer/training/updaters/standard_updater.py:142: FutureWarning: Transfer between @cupy devices was detected and skipped. StandardUpdater normally transfers the model to the specified device, but except for between @cupy devices. That is, if a part of the model is on @cupy:n device and the specified device is @cupy:m device, that part of the model will be left in @cupy:n device. This behavior is planned to be changed in near future. After that, the model will be transferred to the specified device regardless of device combination. If you want to keep the model device but only want to transfer the input data to a given device, specify the 'input_device' argument instead and leave the 'device' argument unspecified.

  FutureWarning)


epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
1           18.9048                           0.705352                                 2420.82       
2           16.5162     15.5985               0.748959       0.76421                   4944.03       
3           15.1737                           0.769564                                 7160.79       
4           14.2003     13.7116               0.78493        0.793349                  9635.98       
5           13.4844                           0.796335                                 11937.9       
6           12.9102     12.6559               0.805611       0.810277                  14285         
7           12.4365                           0.81389                                  16517.4       
8           12.0371     11.8765               0.82162        0.826217                  18946.4       
9           11.6926                           0.82869                               

### 6. Checking the performance with test data

In [ ]:
def evaluate(model, test_trees):
    result = collections.defaultdict(lambda: 0)
    with chainer.using_config('train', False), chainer.no_backprop_mode():
        for tree in test_trees:
            model.traverse(tree, evaluate=result)
    acc_node = 100.0 * result['correct_node'] / result['total_node']
    acc_root = 100.0 * result['correct_root'] / result['total_root']
    print(' Total Node accuracy: {0:.2f} %% ({1:,d}/{2:,d})'.format(
        acc_node, result['correct_node'], result['total_node']))
    print(' Sentence Level (Root) accuracy: {0:.2f} %% ({1:,d}/{2:,d})'.format(
        acc_root, result['correct_root'], result['total_root']))
            
print('Test evaluation')
evaluate(model, test_data)
evaluate(model, validation_data)
evaluate(model, train_data)